In [1]:
import sys
import os
import time

sys.path.insert(0, '..')
sys.path.insert(0, '../../futile/python')

package = 'RemoteDataset'

import BigDFT
BigDFT.Logger.path = os.path.join(os.getcwd(), f'BigDFT_test{package}.yaml')
BigDFT.Logger.level = 'debug'
BigDFT.Logger.overwrite = True
BigDFT.Logger.info(f'{package} unit testing begins')

import BigDFT.RemoteRunners as r

from BigDFT.URL import URL

remote_dir = os.path.join(os.getcwd(), f'fake_remote_{package}')
local_dir = os.path.join(os.getcwd(), f'fake_local_{package}')

print(f'running with:\nremote dir: {remote_dir}\nlocal dir: {local_dir}')

running with:
remote dir: /home/test/bigdft-suite/PyBigDFT/Tests/fake_remote_RemoteDataset
local dir: /home/test/bigdft-suite/PyBigDFT/Tests/fake_local_RemoteDataset


In [2]:
def remote_func(arg):
    """this function should write the argument in the resultfile"""
    return arg

In [3]:
BigDFT.Logger.info('setting up database for checks')
url = URL()
url.test_connection()
url.cmd('rm database.yaml')
remote_data = r.RemoteDataset(function=remote_func,
                              url=url,
                              script='#!/bin/bash',                              
                              remote_dir=remote_dir,
                              run_dir=local_dir,
                              verbose=False,
                              skip=False)


######## local permissions ########
creating test folder /URL_connection_test
	permission denied, trying again in /home/test
creating test folder /home/test/URL_connection_test
	success, cleaning up

######## local permissions ########
creating test folder /URL_connection_test
	permission denied, trying again in /home/test
creating test folder /home/test/URL_connection_test
	success, cleaning up

######## file push test ########
test push testfile_push from /home/test/rsync_test_local to /home/test/rsync_test_local
	success, cleaning up

######## file pull test ########
test pull testfile_pull from /home/test/rsync_test_remote to /home/test/rsync_test_local
	success, cleaning up
used 26 cmd calls for this test, taking 0.10 seconds (averaging 0.00s per call)


In [4]:
BigDFT.Logger.info('appending runs')
arguments = ['Hello World',
             'This is test']
idx=0
for arg in arguments:
    remote_data.append_run(id={'r':idx}, arguments=dict(arg=arg))
    idx+=1

In [5]:
remote_data.run()

{0: None, 1: None}

In [6]:
# NBVAL_CHECK_OUTPUT
idx = 0
maxtime = 20
while not remote_data.all_finished(anyfile=False, timeout=maxtime):
    time.sleep(1)
results = remote_data.fetch_results()

print(results)

['Hello World', 'This is test']


In [7]:
BigDFT.Logger.info('testing run append with identical arguments')

def simple_func(r):
    return r
remote_data = r.RemoteDataset(function = simple_func)

for i in range(3):
    remote_data.append_run(id = {'same': i}, arguments = {'r': 1})

for i in range(2):
    remote_data.append_run(id = {'new': i}, arguments = {'r': i + 5})

In [8]:
# NBVAL_CHECK_OUTPUT
for calc in remote_data.calculators:
    print(calc["runs"])

[0, 1, 2]
[3]
[4]
